## Data Preparation + Feature Engineering

- This notebook consists of two parts. First there will be feature engineering. I will create new variables based on the current variables I have.
- After that I will go ahead and prepare the data for clustering.

In [1]:
# Import libraries

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from pathlib import Path

In [2]:
ds=pd.read_csv('../data/amazon_cleaned.csv')

In [3]:
# Ensuring the dataset was loaded in properly

ds

,OrderID,CustomerID,ProductID,ProductName,Category,Brand,Quantity,UnitPrice,Discount,Tax,ShippingCost,TotalAmount,PaymentMethod,OrderStatus,City,State,SellerID,OrderMonth,OrderYear
0,ORD0000001,CUST001504,P00014,Drone Mini,Electronic,BrightLux,3,65.75,0.00,14.83,11.91,223.99,Debit Card,Delivered,Washington,DC,SELL01967,1,2023
1,ORD0000002,CUST000178,P00040,Microphone,Electronic,UrbanStyle,1,7.38,0.05,0.55,7.85,15.41,Amazon Pay,Delivered,Fort Worth,TX,SELL01298,12,2023
2,ORD0000003,CUST047516,P00044,Power Bank 20000mAh,Electronic,UrbanStyle,3,31.13,0.10,7.02,12.44,103.51,Debit Card,Delivered,Austin,TX,SELL00908,5,2022
3,ORD0000004,CUST030059,P00041,Webcam Full HD,Electronic,Zenith,5,26.21,0.15,9.85,17.34,138.58,Amazon Pay,Delivered,Charlotte,NC,SELL01164,7,2023
4,ORD0000005,CUST048677,P00029,T-Shirt,Clothing,KiddoFun,2,76.28,0.25,11.47,10.74,136.63,Credit Card,Cancelled,San Antonio,TX,SELL01411,2,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,ORD0099996,CUST001356,P00047,Memory Card 128GB,Electronic,Apex,2,62.72,0.00,9.43,7.70,142.57,Debit Card,Delivered,Jacksonville,FL,SELL00041,3,2023
99996,ORD0099997,CUST031254,P00046,Car Charger,Electronic,Apex,5,13.64,0.00,5.13,14.68,88.01,Credit Card,Delivered,San Jose,CA,SELL01449,11,2021
99997,ORD0099998,CUST012579,P00030,Dress Shirt,Clothing,BrightLux,4,47.31,0.00,14.23,13.31,216.78,Debit Card,Delivered,Indianapolis,IN,SELL00028,4,2023
99998,ORD0099999,CUST026243,P00046,Car Charger,Electronic,HomeEase,1,16.87,0.00,1.27,7.75,25.89,Debit Card,Delivered,Charlotte,NC,SELL00324,11,2021


### Feature engineering

I will be creating some new variables outside of the one currently included in the dataset

In [4]:
# DS (For cluster for identifying customer groups

# Creating a copy DS
ds_customers=ds.copy()

# Creating Sunkcost feature
ds_customers['SunkCost']=ds_customers['Tax'] + ds_customers['ShippingCost']


# Aggregating the data to customer level
customer_features=ds_customers.groupby('CustomerID').agg(
    CustomerTotalSpend=('TotalAmount','sum'),
    CustomerOrderCount=('OrderID','count'),
    AvgOrderValue=('TotalAmount','mean'),
    AvgDiscountRate=('Discount', 'mean'),
    AvgQuantity=('Quantity','mean'),
    AvgSunkCost=('SunkCost','mean')).reset_index()

# Rouding to 2 decimals
customer_features=customer_features.round(2)

# Checking the head to make sure everything ran correctly
customer_features.head()


,CustomerID,CustomerTotalSpend,CustomerOrderCount,AvgOrderValue,AvgDiscountRate,AvgQuantity,AvgSunkCost
0,CUST000001,26.72,1,26.72,0.10,1.00,10.57
1,CUST000002,347.29,3,115.76,0.08,3.00,21.24
2,CUST000003,94.71,1,94.71,0.15,5.00,24.71
3,CUST000004,338.47,3,112.82,0.18,2.33,20.04
4,CUST000005,336.13,1,336.13,0.20,4.00,40.42


In [5]:
# Checking the data types
customer_features.dtypes

CustomerID             object
CustomerTotalSpend    float64
CustomerOrderCount      int64
AvgOrderValue         float64
AvgDiscountRate       float64
AvgQuantity           float64
AvgSunkCost           float64
dtype: object

In [6]:
# Ensuring there is no missing
customer_features.isna().sum()

CustomerID            0
CustomerTotalSpend    0
CustomerOrderCount    0
AvgOrderValue         0
AvgDiscountRate       0
AvgQuantity           0
AvgSunkCost           0
dtype: int64

In [7]:
# Copying the dataset. This will add additional features for clustering by seasonality
seasonality_features=customer_features.copy()

# Monthly order counts

Monthly_OCounts=( ds.pivot_table(index='CustomerID',
                                 columns='OrderMonth',
                                 values='OrderID',
                                 aggfunc='count',
                                 fill_value=0))

# Renaming the Monthly Order counts columns

Monthly_OCounts.columns = [f'OrdersInMonth_{m}' for m in Monthly_OCounts.columns]


# Joining the data
seasonality_features=seasonality_features.merge(Monthly_OCounts, on='CustomerID', how='left')

# Monthly Percentages
Monthly_pct=Monthly_OCounts.div(Monthly_OCounts.sum(axis=1), axis=0).fillna(0)
Monthly_pct.columns=[f'PctOrdersInMonth_{m}' for m in Monthly_pct.columns]

# Joining the monthly order percentage
seasonality_features=seasonality_features.merge(Monthly_pct, on='CustomerID', how='left')

# Rounding percentage to 2 decimals
seasonality_features=seasonality_features.round(2)

# Checking the head to make sure everything ran correctly
seasonality_features.head()



,CustomerID,CustomerTotalSpend,CustomerOrderCount,AvgOrderValue,AvgDiscountRate,AvgQuantity,AvgSunkCost,OrdersInMonth_1,OrdersInMonth_2,OrdersInMonth_3,...,PctOrdersInMonth_OrdersInMonth_3,PctOrdersInMonth_OrdersInMonth_4,PctOrdersInMonth_OrdersInMonth_5,PctOrdersInMonth_OrdersInMonth_6,PctOrdersInMonth_OrdersInMonth_7,PctOrdersInMonth_OrdersInMonth_8,PctOrdersInMonth_OrdersInMonth_9,PctOrdersInMonth_OrdersInMonth_10,PctOrdersInMonth_OrdersInMonth_11,PctOrdersInMonth_OrdersInMonth_12
0,CUST000001,26.72,1,26.72,0.10,1.00,10.57,0,0,0,...,0.00,0.0,0.0,0.00,1.00,0.0,0.0,0.0,0.0,0.0
1,CUST000002,347.29,3,115.76,0.08,3.00,21.24,0,1,1,...,0.33,0.0,0.0,0.00,0.33,0.0,0.0,0.0,0.0,0.0
2,CUST000003,94.71,1,94.71,0.15,5.00,24.71,0,0,0,...,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,1.0
3,CUST000004,338.47,3,112.82,0.18,2.33,20.04,0,0,0,...,0.00,0.0,0.0,0.67,0.33,0.0,0.0,0.0,0.0,0.0
4,CUST000005,336.13,1,336.13,0.20,4.00,40.42,0,0,1,...,1.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0


In [8]:
# Checking the data types
seasonality_features.dtypes

CustomerID                            object
CustomerTotalSpend                   float64
CustomerOrderCount                     int64
AvgOrderValue                        float64
AvgDiscountRate                      float64
AvgQuantity                          float64
AvgSunkCost                          float64
OrdersInMonth_1                        int64
OrdersInMonth_2                        int64
OrdersInMonth_3                        int64
OrdersInMonth_4                        int64
OrdersInMonth_5                        int64
OrdersInMonth_6                        int64
OrdersInMonth_7                        int64
OrdersInMonth_8                        int64
OrdersInMonth_9                        int64
OrdersInMonth_10                       int64
OrdersInMonth_11                       int64
OrdersInMonth_12                       int64
PctOrdersInMonth_OrdersInMonth_1     float64
PctOrdersInMonth_OrdersInMonth_2     float64
PctOrdersInMonth_OrdersInMonth_3     float64
PctOrdersI

In [9]:
# Ensuring there is no missings
seasonality_features.isna().sum()

CustomerID                           0
CustomerTotalSpend                   0
CustomerOrderCount                   0
AvgOrderValue                        0
AvgDiscountRate                      0
AvgQuantity                          0
AvgSunkCost                          0
OrdersInMonth_1                      0
OrdersInMonth_2                      0
OrdersInMonth_3                      0
OrdersInMonth_4                      0
OrdersInMonth_5                      0
OrdersInMonth_6                      0
OrdersInMonth_7                      0
OrdersInMonth_8                      0
OrdersInMonth_9                      0
OrdersInMonth_10                     0
OrdersInMonth_11                     0
OrdersInMonth_12                     0
PctOrdersInMonth_OrdersInMonth_1     0
PctOrdersInMonth_OrdersInMonth_2     0
PctOrdersInMonth_OrdersInMonth_3     0
PctOrdersInMonth_OrdersInMonth_4     0
PctOrdersInMonth_OrdersInMonth_5     0
PctOrdersInMonth_OrdersInMonth_6     0
PctOrdersInMonth_OrdersIn

## Preparation

To prepare the data for clustering I will first need to drop the CustomerID columns from their respective datasets. After that I will scale the data

In [10]:
# Dropping CustomerID
customer_features=customer_features.drop(columns=['CustomerID'])

# Ensuring the drop worked
customer_features

,CustomerTotalSpend,CustomerOrderCount,AvgOrderValue,AvgDiscountRate,AvgQuantity,AvgSunkCost
0,26.72,1,26.72,0.10,1.00,10.57
1,347.29,3,115.76,0.08,3.00,21.24
2,94.71,1,94.71,0.15,5.00,24.71
3,338.47,3,112.82,0.18,2.33,20.04
4,336.13,1,336.13,0.20,4.00,40.42
...,...,...,...,...,...,...
43228,1635.38,2,817.69,0.00,5.00,73.79
43229,536.89,1,536.89,0.00,3.00,58.03
43230,299.27,2,149.64,0.00,2.00,19.05
43231,97.07,1,97.07,0.20,2.00,16.01


In [11]:
# Dropping CustomerID for the Seasonality dataset
seasonality_features=seasonality_features.drop(columns=['CustomerID'])

# Ensuring the drop worked
seasonality_features

,CustomerTotalSpend,CustomerOrderCount,AvgOrderValue,AvgDiscountRate,AvgQuantity,AvgSunkCost,OrdersInMonth_1,OrdersInMonth_2,OrdersInMonth_3,OrdersInMonth_4,...,PctOrdersInMonth_OrdersInMonth_3,PctOrdersInMonth_OrdersInMonth_4,PctOrdersInMonth_OrdersInMonth_5,PctOrdersInMonth_OrdersInMonth_6,PctOrdersInMonth_OrdersInMonth_7,PctOrdersInMonth_OrdersInMonth_8,PctOrdersInMonth_OrdersInMonth_9,PctOrdersInMonth_OrdersInMonth_10,PctOrdersInMonth_OrdersInMonth_11,PctOrdersInMonth_OrdersInMonth_12
0,26.72,1,26.72,0.10,1.00,10.57,0,0,0,0,...,0.00,0.0,0.0,0.00,1.00,0.0,0.0,0.0,0.0,0.0
1,347.29,3,115.76,0.08,3.00,21.24,0,1,1,0,...,0.33,0.0,0.0,0.00,0.33,0.0,0.0,0.0,0.0,0.0
2,94.71,1,94.71,0.15,5.00,24.71,0,0,0,0,...,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,1.0
3,338.47,3,112.82,0.18,2.33,20.04,0,0,0,0,...,0.00,0.0,0.0,0.67,0.33,0.0,0.0,0.0,0.0,0.0
4,336.13,1,336.13,0.20,4.00,40.42,0,0,1,0,...,1.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43228,1635.38,2,817.69,0.00,5.00,73.79,1,0,0,0,...,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.5
43229,536.89,1,536.89,0.00,3.00,58.03,0,0,0,0,...,0.00,0.0,0.0,1.00,0.00,0.0,0.0,0.0,0.0,0.0
43230,299.27,2,149.64,0.00,2.00,19.05,0,0,0,0,...,0.00,0.0,0.0,0.00,0.00,0.5,0.5,0.0,0.0,0.0
43231,97.07,1,97.07,0.20,2.00,16.01,0,0,0,0,...,0.00,0.0,0.0,0.00,0.00,1.0,0.0,0.0,0.0,0.0


#### Scaling the datasets

In [12]:
# Running the scale
scaler=StandardScaler()

In [13]:
# Scaling customer features
customer_scaled=scaler.fit_transform(customer_features)

In [14]:
#Scaling Seasonality data
seasonality_scaled=scaler.fit_transform(seasonality_features)

#### Changing the data back to a df so I can export

In [17]:
# Changing customers scaled data to a dataframe

customer_scaled_df=pd.DataFrame(customer_scaled, columns=customer_features.columns)

In [21]:
# Checking that I have a dataframe
customer_scaled_df

,CustomerTotalSpend,CustomerOrderCount,AvgOrderValue,AvgDiscountRate,AvgQuantity,AvgSunkCost
0,-1.214174,-1.044505,-1.356249,0.412296,-1.860116,-1.393696
1,-0.337423,0.546457,-0.671998,0.094130,-0.000458,-0.476196
2,-1.028223,-1.044505,-0.833763,1.207713,1.859200,-0.177815
3,-0.361546,0.546457,-0.694592,1.684962,-0.623443,-0.579382
4,-0.367946,-1.044505,1.021492,2.003129,0.929371,1.173068
...,...,...,...,...,...,...
43228,3.185470,-0.249024,4.722165,-1.178536,1.859200,4.042512
43229,0.181128,-1.044505,2.564284,-1.178536,-0.000458,2.687330
43230,-0.468757,-0.249024,-0.411639,-1.178536,-0.930287,-0.664511
43231,-1.021769,-1.044505,-0.815627,2.003129,-0.930287,-0.925917


In [20]:
# Changing seasonality scaled data to a dataframe

seasonality_scaled_df=pd.DataFrame(seasonality_scaled, columns=seasonality_features.columns)

In [22]:
# Checking I have a dataframe

seasonality_scaled_df

,CustomerTotalSpend,CustomerOrderCount,AvgOrderValue,AvgDiscountRate,AvgQuantity,AvgSunkCost,OrdersInMonth_1,OrdersInMonth_2,OrdersInMonth_3,OrdersInMonth_4,...,PctOrdersInMonth_OrdersInMonth_3,PctOrdersInMonth_OrdersInMonth_4,PctOrdersInMonth_OrdersInMonth_5,PctOrdersInMonth_OrdersInMonth_6,PctOrdersInMonth_OrdersInMonth_7,PctOrdersInMonth_OrdersInMonth_8,PctOrdersInMonth_OrdersInMonth_9,PctOrdersInMonth_OrdersInMonth_10,PctOrdersInMonth_OrdersInMonth_11,PctOrdersInMonth_OrdersInMonth_12
0,-1.214174,-1.044505,-1.356249,0.412296,-1.860116,-1.393696,-0.452415,-0.428299,-0.446383,-0.442042,...,-0.399174,-0.394401,-0.402466,-0.397179,4.288746,-0.404826,-0.395376,-0.394766,-0.390965,-0.398805
1,-0.337423,0.546457,-0.671998,0.094130,-0.000458,-0.476196,-0.452415,2.002348,1.840707,-0.442042,...,1.168632,-0.394401,-0.402466,-0.397179,1.148283,-0.404826,-0.395376,-0.394766,-0.390965,-0.398805
2,-1.028223,-1.044505,-0.833763,1.207713,1.859200,-0.177815,-0.452415,-0.428299,-0.446383,-0.442042,...,-0.399174,-0.394401,-0.402466,-0.397179,-0.398512,-0.404826,-0.395376,-0.394766,-0.390965,4.376985
3,-0.361546,0.546457,-0.694592,1.684962,-0.623443,-0.579382,-0.452415,-0.428299,-0.446383,-0.442042,...,-0.399174,-0.394401,-0.402466,2.806876,1.148283,-0.404826,-0.395376,-0.394766,-0.390965,-0.398805
4,-0.367946,-1.044505,1.021492,2.003129,0.929371,1.173068,-0.452415,-0.428299,1.840707,-0.442042,...,4.351753,-0.394401,-0.402466,-0.397179,-0.398512,-0.404826,-0.395376,-0.394766,-0.390965,-0.398805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43228,3.185470,-0.249024,4.722165,-1.178536,1.859200,4.042512,1.811913,-0.428299,-0.446383,-0.442042,...,-0.399174,-0.394401,-0.402466,-0.397179,-0.398512,-0.404826,-0.395376,-0.394766,-0.390965,1.989090
43229,0.181128,-1.044505,2.564284,-1.178536,-0.000458,2.687330,-0.452415,-0.428299,-0.446383,-0.442042,...,-0.399174,-0.394401,-0.402466,4.384993,-0.398512,-0.404826,-0.395376,-0.394766,-0.390965,-0.398805
43230,-0.468757,-0.249024,-0.411639,-1.178536,-0.930287,-0.664511,-0.452415,-0.428299,-0.446383,-0.442042,...,-0.399174,-0.394401,-0.402466,-0.397179,-0.398512,1.924733,1.986182,-0.394766,-0.390965,-0.398805
43231,-1.021769,-1.044505,-0.815627,2.003129,-0.930287,-0.925917,-0.452415,-0.428299,-0.446383,-0.442042,...,-0.399174,-0.394401,-0.402466,-0.397179,-0.398512,4.254291,-0.395376,-0.394766,-0.390965,-0.398805


In [23]:
data_path=Path("..","Data")


# Exporting to CSV
customer_scaled_df.to_csv(data_path / "Customer_scaled.csv", index=False)

seasonality_scaled_df.to_csv(data_path / "Seasonality_scaled.csv", index=False)